# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [44]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import requests
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [45]:
weatherpy_df = "../starter_code/WeatherPy.csv"

#Desktop/GT-PTCH-DATA-PT-11-2019-U-C/001-Homework/06-Python-APIs/Instructions/starter_code/WeatherPy.csv

weatherpy_data = pd.read_csv(weatherpy_df)

weatherpy_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tasiilaq,65.61,-37.64,270.15,86,100,13.90,GL,1580015430
1,bengkulu,-3.80,102.27,303.64,62,65,2.86,ID,1580015430
2,vaini,-21.20,-175.20,298.15,100,90,4.60,TO,1580015430
3,carbondale,37.73,-89.22,272.15,100,1,1.24,US,1580015218
4,kloulklubed,7.04,134.26,303.15,74,75,5.10,PW,1580015430
...,...,...,...,...,...,...,...,...,...
543,esperanza,19.58,-70.98,297.04,94,75,2.60,DO,1580015488
544,kawalu,-7.38,108.21,301.03,69,75,2.95,ID,1580015488
545,mbuguni,-3.57,36.95,293.15,100,75,7.20,TZ,1580015488
546,agar,23.70,76.02,297.03,25,0,2.53,IN,1580015488


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [46]:
gmaps.configure(api_key=g_key)

In [47]:
#create humidity heatmap with latitude and longitude using gmaps
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    weatherpy_data[['Lat', 'Lng']], weights= weatherpy_data['Humidity'],
    max_intensity=30, point_radius=3.0
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [48]:
#temperature is in Kelvin and Wind Speed is in meters/sec
#max temperature lower than 80 degrees but higher than 70.

best_weather_df = weatherpy_data.copy()
best_weather_df = narrowed_city_df[narrowed_city_df['Max Temp'] >= 294.261]
best_weather_df = narrowed_city_df[narrowed_city_df['Max Temp'] <= 299.817] 

#Wind speed less than 10 mph.
best_weather_df = narrowed_city_df[narrowed_city_df['Wind Speed'] < 4.4704]

#Zero cloudiness.
best_weather_df = narrowed_city_df[narrowed_city_df['Cloudiness'] == 0]

#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

best_weather_df = narrowed_city_df.dropna()
best_weather_df = narrowed_city_df.reset_index()
best_weather_df

,level_0,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,218,arraial do cabo,-22.97,-42.03,297.15,83,0,4.10,BR,1580015370
1,1,275,pakokku,21.33,95.10,296.07,31,0,2.62,MM,1580015459
2,2,288,mangrol,21.12,70.12,296.27,50,0,2.26,IN,1580015460
3,3,300,san jorge,-31.90,-61.86,295.93,77,0,2.17,AR,1580015462
4,4,327,pandamatenga,-18.53,25.63,296.21,63,0,2.80,BW,1580015465
5,5,348,dwarka,22.24,68.97,295.73,72,0,3.68,IN,1580015467
6,6,523,presidencia roque saenz pena,-26.79,-60.44,294.98,58,0,0.25,AR,1580015485
7,7,546,agar,23.70,76.02,297.03,25,0,2.53,IN,1580015488


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [49]:
# create hotel_df with hotel name column
hotel_df = best_weather_df
hotel_df["Hotel Name"] = np.nan
hotel_df = hotel_df.drop(columns='index')
hotel_df = hotel_df.drop(columns='level_0')
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,arraial do cabo,-22.97,-42.03,297.15,83,0,4.10,BR,1580015370,NaN
1,pakokku,21.33,95.10,296.07,31,0,2.62,MM,1580015459,NaN
2,mangrol,21.12,70.12,296.27,50,0,2.26,IN,1580015460,NaN
3,san jorge,-31.90,-61.86,295.93,77,0,2.17,AR,1580015462,NaN
4,pandamatenga,-18.53,25.63,296.21,63,0,2.80,BW,1580015465,NaN
5,dwarka,22.24,68.97,295.73,72,0,3.68,IN,1580015467,NaN
6,presidencia roque saenz pena,-26.79,-60.44,294.98,58,0,0.25,AR,1580015485,NaN
7,agar,23.70,76.02,297.03,25,0,2.53,IN,1580015488,NaN


In [53]:
#Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
# geocoordinates
hotels = []
x = 0

for cities in hotel_df['City']:
    target_coordinates = str(hotel_df['Lat'][x])+' , '+str(hotel_df['Lng'][x])
    x+=1
    target_radius = 5000
    target_type = "lodging"

# set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}

# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    try:
        # run a request using our params dictionary
        response = requests.get(base_url, params=params)
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name'])
        
    except:
        hotels.append(np.nan)
# Print the results 
hotel_df['Hotel Name'] = hotels
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,arraial do cabo,-22.97,-42.03,297.15,83,0,4.10,BR,1580015370,Pousada Porto Praia
1,pakokku,21.33,95.10,296.07,31,0,2.62,MM,1580015459,Royal Palace Motel
2,mangrol,21.12,70.12,296.27,50,0,2.26,IN,1580015460,Hotel The Royal Honours
3,san jorge,-31.90,-61.86,295.93,77,0,2.17,AR,1580015462,INDIGO Hostel
4,pandamatenga,-18.53,25.63,296.21,63,0,2.80,BW,1580015465,Wildtrack Safaris Eco Lodge
5,dwarka,22.24,68.97,295.73,72,0,3.68,IN,1580015467,Hotel Narayan Avezika Comfort
6,presidencia roque saenz pena,-26.79,-60.44,294.98,58,0,0.25,AR,1580015485,Hotel Aconcagua
7,agar,23.70,76.02,297.03,25,0,2.53,IN,1580015488,Oswal bhawan prakash guruji chomasa


In [55]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [56]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig
# Display Map

Figure(layout=FigureLayout(height='420px'))

In [58]:
plt.savefig("Ideal Weather Conditions.png")

<Figure size 432x288 with 0 Axes>